# 1. install libaries

In [20]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install XBRL

Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install oauth2

     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# 2. import libraries

In [8]:
import requests

In [9]:
import os, re, sys, json

In [10]:
import pandas as pd

In [11]:
import numpy as np

In [12]:
import getpass

# 3. Authentication for xbrl token

### 3.1. acess token

In [13]:
"""
get access token from XBRL API
"""
# print('Enter XBRL US Web account email: ')
email = 'jesse.c.jia@gmail.com'
# print('Account password: ')
password = 'QAZwsx123'
# print('Client ID: ')
clientid = 'bf3ba33f-e98b-4159-aea2-466445f9621f'
# print('Client secret: ')
secret = 'a3f87777-0086-4208-8d3b-df494408271e'

body_auth = {'username' : ''.join(email), 
            'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }
endpoint_auth = 'https://api.xbrl.us/oauth2/token'
res = requests.post(endpoint_auth, data=body_auth)
auth_json = res.json()

if 'error' in auth_json:
        print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
        print ("\n\nYour access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. \n\nFor now, skip ahead to the section 'Make a Query'.")

access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''
    
    



Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.


### 3.2. refresh token

In [48]:
"""
reconnect to XBRL API when expire
"""
token = token if newrefresh != '' else refresh_token 

refresh_auth = {'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'grant_type' : 'refresh_token', 
            'platform' : 'ipynb', 
            'refresh_token' : ''.join(token) }
refreshres = requests.post(endpoint_auth, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)

Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.
a490322d-46a1-4338-b1bb-21afbfc7681a


# 4. Prepare Data

### 4.1 Company CIK file

In [14]:
"""
get document path
"""
root_path = os.path.abspath(os.path.dirname(os.getcwd()))
data_path = os.path.join(root_path, 'data')
company_path = os.path.join(data_path, 'company')
company_cik_path = os.path.join(company_path, 'CIK_Summary.csv')

In [15]:
"""
import company-cik file
"""

#read cik.csv as a pandas DataFrame
ciksDF = pd.read_csv(company_cik_path, header=None)

#set the column with all the tickers as the index
ciksDF = ciksDF.set_index(0)

In [16]:
#function to return CIK of a ticker
def getCIK(ticker):
    """
    input ticker
    return respect cik
    """
    try:
        return int(ciksDF.loc[ticker.lower(), 1])

    #return None there's no CIK for the ticker
    except:
        print(f"{ticker} does not exist.")
        return None
    
    
#function to return CIK of a list of tickers
def outputcik(TICKERS): 
    """
    input ticker list
    return cik repsectively in list
    """
    cikDir = []
    for ticker in TICKERS:
        a = str(getCIK(ticker))
        if a != 'None':
            cik = a.zfill(10)
            cikDir.append(cik)
        else:
            pass
    return cikDir

In [17]:
# test code
lis = outputcik(["uytuyfd","a"])
lis

uytuyfd does not exist.


['0001090872']

### 4.2. Read Company List

In [18]:
# company list

company_list = ["YTEN", "UAN", "CGA", "RKDA", "FMC", "MOS", "SEED", "CTVA", "MBII", "AVD", "ICL", "IPI", "NTR", "MGPI", "SMG", "CF"]

In [19]:
len(company_list)

16

### 4.3 convert company_name into cik

In [20]:
# generate cik list
cik_list = outputcik(company_list)

In [21]:
# cik_list1 = cik_list.pop(2)

In [22]:
len(cik_list)

16

In [23]:
cik_list

['0001121702',
 '0001425292',
 '0000857949',
 '0001469443',
 '0000037785',
 '0001285785',
 '0001321851',
 '0001755672',
 '0001441693',
 '0000005981',
 '0000941221',
 '0001421461',
 '0001725964',
 '0000835011',
 '0000825542',
 '0001324404']

### 4.4 financial data conceptual name

In [24]:
# financial data with the conceptual name on SEC website respectively
Financial_data_dic = {
    "revenue" : "RevenueFromContractWithCustomerExcludingAssessedTax",
    "net_income" : "NetIncomeLoss",
    "operating_income" : "OperatingIncomeLoss", 
    "asset" : "Assets",
    "current_asset" : "AssetsCurrent",
    "inventory" : "InventoryNet",
    "current_liability" : "LiabilitiesCurrent",
    "debt" : "LongTermDebtNoncurrent",
    "equity" : "StockholdersEquity",
    "eps" : "EarningsPerShareBasic"
}

# 5. Request data via XBRL

In [25]:
def acquire_data(company_cik, financial_data):
    """
    input company cik value and financial conceptual name on SEC
    return data frame of financial value, year, entity name, and so on, (details see fiels list)
    """
    
    # Define the fields to be returned
    fields = ['period.fiscal-year.sort(DESC)',
              'period.end.sort(DESC)',
              'entity.name.sort(ASC)',
              'entity.cik.sort(ASC)',
              'concept.local-name.sort(ASC)',
              'fact.value',
              'unit',
              'report.filing-date']
    
    # Filter the query for specific XBRL elements (tags) to research
    XBRL_Elements = [financial_data
                ]
    
    # Filter companies for the query
    companies_cik = [company_cik
                ]
    
    # api parameters
    params = {'fields': ','.join(fields),
         'concept.local-name': ','.join(XBRL_Elements),
         'entity.cik': ','.join(companies_cik),
         #'period.fiscal-year': ','.join(years),
         'period.fiscal-period': 'Y',
         'fact.has-dimensions': 'FALSE',
         'fact.ultimus': 'TRUE'
         }
    
    search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
    res_json = res.json()
    res_df = pd.DataFrame(res_json['data'])
    
#     print(res_json['paging'])
#     print(res_df)
    return res_df

In [26]:
def bigger(a, b):
    if a >= b:
        return a
    else:
        return b

def smaller(a,b):
    if a <= b:
        return a
    else:
        return b


In [27]:

len(Financial_data_dic)

10

In [28]:
# for cik in cik_list:
#     #print(cik)
#     cik_fin_value_df = df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
#     for k, v in Financial_data_dic.items():
#         print(cik,v)
#         temp_df = acquire_data(cik, v) asset
#         print(temp_df)
        




In [29]:
def generate_financial_data(cik, Financial_data_dic):
    """
    input company's cik value and financial item dic
    acquire financial value through xrbl API,
    calculate financial ratio and append in data frame
    return a data frame with year, company name, financial ratios
    """
    
    # create a empty data frame for use
    cik_fin_value_df = pd.DataFrame()
    # cik = "0001121702"
    for k, v in Financial_data_dic.items():
        print(f"current cik is {cik}, acquire {k}")
        
        # return first financial values into df
        if k == "revenue":
            
            cik_fin_value_df = acquire_data(cik, v)
            if cik_fin_value_df.empty:
                pass
            else:
                cik_fin_value_df.rename(columns = {"fact.value" : k}, inplace = True)
                cik_fin_value_df[k] = pd.to_numeric(cik_fin_value_df[k])
                cik_fin_value_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "unit", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "report.filing-date", axis = 1, inplace = True)

    #         cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)

        
        # create temp df, and acquire rest financial data, then concate with inital df
        else:
#             print(cik,k,v)
            temp_df = acquire_data(cik, v)
            if temp_df.empty:
                temp_df[k] = np.NaN
                cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
            else:
                temp_df.rename(columns = {"fact.value" : k}, inplace = True)
                temp_df[k] = pd.to_numeric(temp_df[k])

                temp_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
    #             temp_df.drop(labels = "period.end", axis = 1, inplace = True)
                temp_df.drop(labels = "unit", axis = 1, inplace = True)
                temp_df.drop(labels = "report.filing-date", axis = 1, inplace = True)
    #             temp_df.drop(labels = "entity.name", axis = 1, inplace = True)


    #             if len(temp_df["period.end"]) >= len(cik_fin_value_df["period.end"]):
    #                 cik_fin_value_df.drop(labels = "period.end", axis = 1, inplace = True)
    #             else:
                temp_df.drop(labels = "period.end", axis = 1, inplace = True)
    
                # keep the most company name column
                try:
                    if len(temp_df["entity.cik"]) >= len(cik_fin_value_df["entity.cik"]):
                        cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        cik_fin_value_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                    else:
                        temp_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        temp_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                except:
                    pass



    #             cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
    #             cikabs_fin_value_df = pd.concat((cik_fin_value_df, temp_df), axis = 1, join = "outer")
                try:
                    cik_fin_value_df = pd.merge(cik_fin_value_df, temp_df, on = "period.fiscal-year", how = "outer")
                except:
                    pass
    
    # calculate financial ratios
#     Ratios_calculator(cik_fin_value_df)
    print(f"{cik} fin data acquire compeleted.")
    
    return cik_fin_value_df



# cik_fin_value_df["ROA1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)
# cik_fin_value_df["ROE1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)
# cik_fin_value_df["ROC1"] = cik_fin_value_df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)
# cik_fin_value_df["OperatingMargin"] = cik_fin_value_df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["NetMargin"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["CurrentRatio"] = cik_fin_value_df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)
# cik_fin_value_df["QuickRatio"] = cik_fin_value_df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)
# cik_fin_value_df["DERatio"] = cik_fin_value_df.apply(lambda x:x["debt"]/x["equity"], axis = 1)



In [30]:
def ROA1(df):
    df["ROA1"] = df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)

def ROE1(df):
    df["ROE1"] = df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)

def ROC1(df):
    df["ROC1"] = df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)

def Op_Margin(df):
    df["OperatingMargin"] = df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
    
def Net_Margin(df):
    df["NetMargin"] = df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
    
def Current_Ratio(df):
    df["CurrentRatio"] = df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)

def Quick_Ratio(df):
    df["QuickRatio"] = df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)

def DE(df):
    df["DERatio"] = df.apply(lambda x:x["debt"]/x["equity"], axis = 1)


In [31]:
def Ratios_calculator(df):
    ROA1(df)
    ROE1(df)
    ROC1(df)
    Op_Margin(df)
    Net_Margin(df)
    Current_Ratio(df)
    Quick_Ratio(df)
    DE(df)
    
    return df

In [32]:
def generate_financial_ratio(cik, fin_data_dic, valid_list, invalid_list):
    df = generate_financial_data(cik, fin_data_dic)
    #print(cik)

    try:
        Ratios_calculator(df)
        #print(df)
        print(f"{cik} fin ratios calculated")
        valid_list.append(cik)
    except:
        print(f"{cik} has no ratios return")
        invalid_list.append(cik)
        pass
    
    return df, valid_list, invalid_list

In [33]:
def output_file(cik_list, fin_data_dic, industry):
    industry_df = pd.DataFrame()
    valid_company_list = []
    invalid_company_list = []
    for c in cik_list:
        temp_df, valid_list, invalid_list = generate_financial_ratio(c, fin_data_dic, valid_company_list, invalid_company_list)
        
        industry_df = pd.concat([industry_df, temp_df], axis = 0)
        
#         print(f"{c} has been added.")
    
    industry_df.to_csv(f"{data_path}/{industry}.csv")
    
#     print(f"The list cik companies have connection with SEC website: {valid_company_list}.")
#     print(f"The list cik companies have no connection with SEC website, no value returned by using XBEL: {valid_company_list}.")
    
    return industry_df, valid_company_list, invalid_company_list

# 5. output process

In [34]:
indus, valid_company_list, invalid_company_list = output_file(cik_list, Financial_data_dic, "agriculture")

current cik is 0001121702, acquire revenue
current cik is 0001121702, acquire net_income
current cik is 0001121702, acquire operating_income
current cik is 0001121702, acquire asset
current cik is 0001121702, acquire current_asset
current cik is 0001121702, acquire inventory
current cik is 0001121702, acquire current_liability
current cik is 0001121702, acquire debt
current cik is 0001121702, acquire equity
current cik is 0001121702, acquire eps
0001121702 fin data acquire compeleted.
0001121702 fin ratios calculated
current cik is 0001425292, acquire revenue
current cik is 0001425292, acquire net_income
current cik is 0001425292, acquire operating_income
current cik is 0001425292, acquire asset
current cik is 0001425292, acquire current_asset
current cik is 0001425292, acquire inventory
current cik is 0001425292, acquire current_liability
current cik is 0001425292, acquire debt
current cik is 0001425292, acquire equity
current cik is 0001425292, acquire eps
0001425292 fin data acquire

/Users/j/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


current cik is 0000857949, acquire net_income
current cik is 0000857949, acquire operating_income
current cik is 0000857949, acquire asset
current cik is 0000857949, acquire current_asset
current cik is 0000857949, acquire inventory
current cik is 0000857949, acquire current_liability
current cik is 0000857949, acquire debt
current cik is 0000857949, acquire equity
current cik is 0000857949, acquire eps
0000857949 fin data acquire compeleted.
0000857949 has no ratios return
current cik is 0001469443, acquire revenue
current cik is 0001469443, acquire net_income
current cik is 0001469443, acquire operating_income
current cik is 0001469443, acquire asset
current cik is 0001469443, acquire current_asset
current cik is 0001469443, acquire inventory
current cik is 0001469443, acquire current_liability
current cik is 0001469443, acquire debt
current cik is 0001469443, acquire equity
current cik is 0001469443, acquire eps
0001469443 fin data acquire compeleted.
0001469443 fin ratios calculate

In [35]:
print(f"The list cik companies have connection with SEC website:")
for cik in valid_company_list:
    print(cik)


The list cik companies have connection with SEC website:
0001121702
0001425292
0001469443
0000037785
0001285785
0001441693
0000005981
0000835011
0000825542


In [36]:
print(f"The list cik companies have no connection with SEC website, no return data via XBRL:")
for cik in invalid_company_list:
    print(cik)

The list cik companies have no connection with SEC website, no return data via XBRL:
0000857949
0001321851
0001755672
0000941221
0001421461
0001725964
0001324404


In [37]:
def save_csv(file, path, industry):
    file.to_csv(f"{path}/{industry}.csv")
    print(f"{file} has been saved as csv.")

In [38]:
save_csv(indus, data_path, "agriculture")

    CurrentRatio   DERatio  NetMargin  OperatingMargin  QuickRatio      ROA1  \
0       5.701213       NaN -12.773467       -12.026283         NaN -0.702796   
1       5.294064       NaN -16.074442       -10.665012         NaN -0.774973   
2       3.886045       NaN -16.519784       -16.760791         NaN -0.708719   
3       6.280842       NaN  -9.953390        -9.833686         NaN -0.556569   
4       2.849891       NaN        NaN              NaN    2.849891 -0.707480   
..           ...       ...        ...              ...         ...       ...   
16      2.118011  1.414434        NaN              NaN    1.292147  0.081815   
17      1.517508  0.571223        NaN              NaN    1.038545  0.094316   
18      1.441464       NaN        NaN              NaN    0.835095  0.069051   
19           NaN       NaN        NaN              NaN         NaN       NaN   
20           NaN       NaN        NaN              NaN         NaN       NaN   

        ROC1      ROE1         asset  c